In [21]:
#referred to lesson 9.05
#https://stackoverflow.com/questions/61757170/python-unstacked-dataframe-is-too-big-causing-int32-overflow
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity

In [2]:
df=pd.read_csv('./data/video_games.csv')

In [3]:
df.isnull().sum()

customer_id           0
review_id             0
product_id            0
product_parent        0
product_title         0
star_rating           0
helpful_votes         0
total_votes           0
verified_purchase     0
review_date           0
full_review          47
dtype: int64

In [4]:
df.dropna(inplace=True)

In [5]:
df.isnull().sum()

customer_id          0
review_id            0
product_id           0
product_parent       0
product_title        0
star_rating          0
helpful_votes        0
total_votes          0
verified_purchase    0
review_date          0
full_review          0
dtype: int64

In [6]:
df.head()

,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,verified_purchase,review_date,full_review
0,12039526,RTIS3L2M1F5SM,B001CXYMFS,737716809,Thrustmaster T-Flight Hotas X Flight Stick,5,0,0,1,2015-08-31,amazing joystick I especially love twist Used ...
1,2331478,R3BH071QLH8QMC,B0029CSOD2,98937668,Hidden Mysteries: Titanic Secrets of the Fatef...,1,0,1,1,2015-08-31,One Star poor quality work advertised
2,52495923,R127K9NTSXA2YH,B00GOOSV98,23143350,GelTabz Performance Thumb Grips - PlayStation ...,3,0,0,1,2015-08-31,good could bettee nice tend slip away stick in...
3,14533949,R32ZWUXDJPW27Q,B00Y074JOM,821342511,Zero Suit Samus amiibo - Japan Import (Super S...,4,0,0,1,2015-08-31,Great flawed Great amiibo great collecting Qua...
4,17521011,R2F0POU5K6F73F,B008XHCLFO,24234603,Protection for your 3DS XL,5,0,0,1,2015-08-31,A Must I 2012 2013 XL durable comfortable real...


In [9]:
df.columns

Index(['customer_id', 'review_id', 'product_id', 'product_parent',
       'product_title', 'star_rating', 'helpful_votes', 'total_votes',
       'verified_purchase', 'review_date', 'full_review'],
      dtype='object')

In [18]:
df2 = df[['customer_id', 'product_id', 'product_title', 'star_rating']].copy()

#pivot = pd.pivot_table(df2, index='product_title', columns='customer_id', values='star_rating')

My first attempt at making a pivot table from this data failed because the created dataframe is too big. I'll have to do it in chunks.

In [13]:
chunk_sz = 10_000
chunks = [n for n in range(0, len(df2), chunk_sz)]

In [14]:
ch_df = pd.concat(
    [df2.loc[chunks[i]:chunks[i+1]-1].pivot(
        index='product_title', columns='customer_id', values='star_rating')
    for i in range(0,len(chunks)-1)])

ValueError: Index contains duplicate entries, cannot reshape